#### Declarative Table

With the `declarative base class`, the _typical form of mapping_ includes an attribute `__tablename__` that indicates the name of a `Table` that should be __generated along with the mapping__.

In [1]:
from sqlalchemy import Column, Integer, String, Table, inspect, create_engine
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

Above, `Column` objects are __placed inline__ with the class definition. The `declarative mapping process` will _generate_ a new_ `Table` _object_ against the __`MetaData` collection associated with the `declarative base`__, and each specified `Column` object will become part of the __`Table.columns` collection__ of this `Table` object. The `Column` objects can omit their `"name"` field, which is usually the _first positional argument_ to the `Column` constructor; the `declarative system` will assign the __key associated with each `Column`__ as the `name`, to produce a `Table` that is equivalent to:

In [4]:
# equivalent Table object produced
user_table = Table(
    "equivalent_user",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
    Column("fullname", String),
    Column("nickname", String),
)

#### Accessing Table and Metadata

A `declaratively mapped class` will __always include__ an attribute called `__table__`; when the above _configuration_ using `__tablename__` is __complete__, the _declarative process_ makes the `Table` available via the `__table__` attribute.

In [5]:
user_table = User.__table__
print(f"{user_table = }")

user_table = Table('user', MetaData(), Column('id', Integer(), table=<user>, primary_key=True, nullable=False), Column('name', String(), table=<user>), Column('fullname', String(), table=<user>), Column('nickname', String(), table=<user>), schema=None)


The above table is __ultimately the same__ one that _corresponds_ to the `Mapper.local_table` attribute, which we can see through the `runtime inspection system`.

In [6]:
user_table = inspect(User).local_table
print(f"{user_table = }")

user_table = Table('user', MetaData(), Column('id', Integer(), table=<user>, primary_key=True, nullable=False), Column('name', String(), table=<user>), Column('fullname', String(), table=<user>), Column('nickname', String(), table=<user>), schema=None)


The `MetaData` collection _associated_ with both the `declarative registry` as well as the `base class` is __frequently necessary__ in order to _run DDL operations_ such as `CREATE`, as well as in _use with migration tools_ such as [`Alembic`](https://alembic.sqlalchemy.org/en/latest/index.html). This object is available via the `.metadata` attribute of `registry` as well as the `declarative base class`. Below, for a small script we may wish to _emit_ a `CREATE` for all tables against a `SQLite database`.

In [7]:
engine = create_engine("sqlite://")

Base.metadata.create_all(engine)